In [1]:
import pandas as pd
import hopsworks
import joblib
import datetime
from PIL import Image
from datetime import datetime
import dataframe_image as dfi
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot
import seaborn as sns
import requests

In [2]:
project = hopsworks.login(api_key_value='meVTnNcyrLQJefeF.OcatdkM3qucgCg0WvGCRD81QRGyrgxZl1RgPtiKqNcA9gSwsNn1TIEWnTeD9c31n')
fs = project.get_feature_store()
mr = project.get_model_registry()
model = mr.get_model("wine_model", version=1)
model_dir = model.download()
model = joblib.load(model_dir + "/wine_model.pkl")

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/196758
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [8]:
feature_view = fs.get_feature_view(name="wine", version=1)
batch_data = feature_view.get_batch_data()
batch_data

Finished: Reading data from Hopsworks, using ArrowFlight (1.18s) 


,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,density,ph,sulphates,alcohol,quality
0,red,11.8,0.330,0.490,3.4,0.0930,54.0,1.00020,3.30,0.76,10.70,7
1,white,7.5,0.420,0.190,6.9,0.0410,62.0,0.99508,3.23,0.37,10.00,6
2,white,7.5,0.240,0.620,10.6,0.0450,51.0,0.99779,3.16,0.44,8.80,5
3,red,8.6,0.685,0.100,1.6,0.0920,3.0,0.99745,3.31,0.65,9.55,6
4,white,6.6,0.500,0.260,11.3,0.0290,32.0,0.99302,3.27,0.78,12.90,8
...,...,...,...,...,...,...,...,...,...,...,...,...
5282,red,7.0,0.580,0.280,4.8,0.0850,12.0,0.99633,3.32,0.70,11.00,6
5283,white,6.7,0.210,0.340,1.4,0.0490,36.0,0.99091,3.02,0.50,11.00,6
5284,white,5.4,0.375,0.400,3.3,0.0540,29.0,0.99482,3.42,0.52,9.10,5
5285,red,9.3,0.775,0.270,2.8,0.0780,24.0,0.99840,3.31,0.67,10.60,6


In [ ]:
y_pred = model.predict(batch_data)
#print(y_pred)
offset = 1
wine = y_pred[y_pred.size-offset]
#wine_url = "https://raw.githubusercontent.com/featurestoreorg/serverless-ml-course/main/src/01-module/assets/" + wine + ".png"
print("wine predicted: " + wine)
#img = Image.open(requests.get(wine_url, stream=True).raw)            
#img.save("./latest_wine.png")
dataset_api = project.get_dataset_api()    
#dataset_api.upload("./latest_wine.png", "Resources/images", overwrite=True)

wine_fg = fs.get_feature_group(name="wine", version=1)
df = wine_fg.read() 
#print(df)
label = df.iloc[-offset]["quality"]
#label_url = "https://raw.githubusercontent.com/featurestoreorg/serverless-ml-course/main/src/01-module/assets/" + label + ".png"
print("wine quality actual: " + label)
#img = Image.open(requests.get(label_url, stream=True).raw)            
#img.save("./actual_wine.png")
#dataset_api.upload("./actual_wine.png", "Resources/images", overwrite=True)

monitor_fg = fs.get_or_create_feature_group(name="wine_predictions",
                                            version=1,
                                            primary_key=["datetime"],
                                            description="wine quality Prediction/Outcome Monitoring"
                                            )

now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
data = {
    'prediction': [wine],
    'label': [label],
    'datetime': [now],
    }
monitor_df = pd.DataFrame(data)
monitor_fg.insert(monitor_df, write_options={"wait_for_job" : False})

history_df = monitor_fg.read()
# Add our prediction to the history, as the history_df won't have it - 
# the insertion was done asynchronously, so it will take ~1 min to land on App
history_df = pd.concat([history_df, monitor_df])


# df_recent = history_df.tail(4)
# dfi.export(df_recent, './df_recent.png', table_conversion = 'matplotlib')
# dataset_api.upload("./df_recent.png", "Resources/images", overwrite=True)

predictions = history_df[['prediction']]
labels = history_df[['label']]

# Only create the confusion matrix when our wine_predictions feature group has examples of all 7 quality
print("Number of different wine predictions to date: " + str(predictions.value_counts().count()))
if predictions.value_counts().count() == 7:
    results = confusion_matrix(labels, predictions)

    df_cm = pd.DataFrame(results, ['True Setosa', 'True Versicolor', 'True Virginica'],
                            ['Pred Setosa', 'Pred Versicolor', 'Pred Virginica'])

    cm = sns.heatmap(df_cm, annot=True)
    fig = cm.get_figure()
    fig.savefig("./confusion_matrix.png")
    dataset_api.upload("./confusion_matrix.png", "Resources/images", overwrite=True)
else:
    print("You need 7 different wine quality predictions to create the confusion matrix.")
    print("Run the batch inference pipeline more times until you get 7 different wine quality predictions")